In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Stomach_Cancer"
cohort = "GSE98708"

# Input paths
in_trait_dir = "../../input/GEO/Stomach_Cancer"
in_cohort_dir = "../../input/GEO/Stomach_Cancer/GSE98708"

# Output paths
out_data_file = "../../output/preprocess/Stomach_Cancer/GSE98708.csv"
out_gene_data_file = "../../output/preprocess/Stomach_Cancer/gene_data/GSE98708.csv"
out_clinical_data_file = "../../output/preprocess/Stomach_Cancer/clinical_data/GSE98708.csv"
json_path = "../../output/preprocess/Stomach_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE98708_family.soft.gz', 'GSE98708_series_matrix.txt.gz']
Identified SOFT files: ['GSE98708_family.soft.gz']
Identified matrix files: ['GSE98708_series_matrix.txt.gz']

Background Information:
!Series_title	"Expression profiling of frozen primary and patient derived xenograft gastric cancer"
!Series_summary	"Expression profiling of frozen primary and patient derived xenograft gastric cancer"
!Series_overall_design	"Expression profiling of frozen primary and patient derived xenograft gastric cancer"

Sample Characteristics Dictionary:
{0: ['tissue: gastric cancer'], 1: ['sample type: PDX', 'sample type: primary tumor'], 2: ['patient id: GTR0222', 'patient id: GTR0227', 'patient id: GTR0230', 'patient id: GTR0233', 'patient id: GTR0244', 'patient id: GTR0245', 'patient id: GTR0247', 'patient id: GTR0249', 'patient id: GTR0255', 'patient id: GTR0259', 'patient id: GTR0263', 'patient id: GTR0220', 'patient id: GTR0102', 'patient id: GTR0103', 'patient id: 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information and summary, the dataset appears to be expression profiling of gastric cancer
# samples, suggesting gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait (Stomach Cancer)
# Looking at the sample characteristics, row 0 contains 'tissue: gastric cancer'
# This confirms all samples are gastric cancer tissue
trait_row = 0

# Age data is not available in the sample characteristics dictionary
age_row = None

# Gender data is not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert the trait value to binary (1 for gastric cancer, 0 for normal)"""
    if value is None:
        return None
    
    # Handle non-string types
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Check if it's related to gastric cancer
    if 'gastric cancer' in value:
        return 1
    elif 'normal' in value:
        return 0
    else:
        return None

def convert_age(value):
    """Function to convert age to continuous value (not used in this dataset)"""
    if value is None:
        return None
    
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Function to convert gender to binary (0 for female, 1 for male) (not used in this dataset)"""
    if value is None:
        return None
    
    if not isinstance(value, str):
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value in ['male', 'm']:
        return 1
    elif value in ['female', 'f']:
        return 0
    else:
        return None

# 3. Save Metadata
# is_trait_available is determined by whether trait_row is None
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    try:
        # Load the clinical data from the provided dictionary instead of parsing the file again
        # Use whatever clinical_data source that was provided in the previous step
        # For this dataset, we know the clinical characteristics from the dictionary already shown
        clinical_data = pd.DataFrame()
        
        # Add the sample characteristic row for trait (gastric cancer)
        clinical_data.loc[trait_row, 0] = 'tissue: gastric cancer'
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        # If clinical data processing fails, update metadata
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )


Preview of selected clinical data:
{0: [1.0]}
Clinical data saved to ../../output/preprocess/Stomach_Cancer/clinical_data/GSE98708.csv


### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data shape: (47323, 102)


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (ILMN_) are Illumina probe IDs, not standard human gene symbols.
# They need to be mapped to official gene symbols for proper biological interpretation.
# ILMN_ prefix indicates these are Illumina BeadArray probe identifiers.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
try:
    # Use the correct variable name from previous steps
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # 2. Preview the gene annotation dataframe
    print("Gene annotation preview:")
    print(preview_df(gene_annotation))
    
except UnicodeDecodeError as e:
    print(f"Unicode decoding error: {e}")
    print("Trying alternative approach...")
    
    # Read the file with Latin-1 encoding which is more permissive
    import gzip
    import pandas as pd
    
    # Manually read the file line by line with error handling
    data_lines = []
    with gzip.open(soft_file_path, 'rb') as f:
        for line in f:
            # Skip lines starting with prefixes we want to filter out
            line_str = line.decode('latin-1')
            if not line_str.startswith('^') and not line_str.startswith('!') and not line_str.startswith('#'):
                data_lines.append(line_str)
    
    # Create dataframe from collected lines
    if data_lines:
        gene_data_str = '\n'.join(data_lines)
        gene_annotation = pd.read_csv(pd.io.common.StringIO(gene_data_str), sep='\t', low_memory=False)
        print("Gene annotation preview (alternative method):")
        print(preview_df(gene_annotation))
    else:
        print("No valid gene annotation data found after filtering.")
        gene_annotation = pd.DataFrame()
    
except Exception as e:
    print(f"Error extracting gene annotation data: {e}")
    gene_annotation = pd.DataFrame()


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observing the gene identifiers and annotation data:
# In gene expression data, the indices are in the format "ILMN_XXXXXXX"
# In the gene annotation data, the "ID" column contains this same identifier format
# The "Symbol" column appears to contain the gene symbols we need to map to

# 2. Extract the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print a preview of the mapping
print("Gene mapping preview (probe ID to gene symbol):")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This divides probe values among multiple genes and sums contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the resulting gene expression data
print("\nGene expression data after mapping (first few genes):")
print(preview_df(gene_data))
print(f"Mapped gene data shape: {gene_data.shape}")

# Normalize gene symbols (handle case variations and synonyms)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nNormalized gene data shape: {gene_data.shape}")

# Save the processed gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview (probe ID to gene symbol):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Gene expression data after mapping (first few genes):
{'GSM2610417': [246.4, 340.3, 267.6, 398.4, 207.5], 'GSM2610418': [246.60000000000002, 324.1, 333.3, 392.0, 95.3], 'GSM2610419': [223.3, 330.7, 280.1, 484.2, 137.9], 'GSM2610420': [262.1, 1330.2, 291.9, 369.7, 358.5], 'GSM2610421': [229.5, 1021.2, 396.3, 422.7, 367.4], 'GSM2610422': [281.20000000000005, 344.9, 386.6, 398.79999999999995, 322.2], 'GSM2610423': [235.0, 1637.0, 311.1, 352.4, 587.1], 'GSM2610424': [241.89999999999998, 858.5, 324.29999999999995, 348.7, 718.5], 'GSM2610425': [247.5, 1440.1999999999998, 364.0, 389.0, 575.5], 'GSM2610426': [265.8, 310.8, 247.7, 336.7, 1204.7], 'GSM2610427': [254.79999999999998, 732.6, 290.6, 354.9, 715.8], 'GSM2610428': [258.5, 323.3, 238.6


Normalized gene data shape: (20259, 102)


Gene expression data saved to ../../output/preprocess/Stomach_Cancer/gene_data/GSE98708.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
# (This step was already completed in the previous step)
print(f"Normalized gene data shape: {gene_data.shape}")
print(f"First few normalized gene symbols: {list(gene_data.index[:10])}")

# 2. Load the previously saved clinical data and prepare it for linking
try:
    # Read the clinical CSV file
    clinical_data = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_data.shape}")
    
    # Create properly structured clinical features for linking
    # The geo_select_clinical_features function should have created a DataFrame with traits as rows
    # But we'll verify and fix the structure if needed
    if trait not in clinical_data.columns:
        # Create a properly formatted clinical features DataFrame with trait as row
        clinical_features = pd.DataFrame(index=[trait], data=[clinical_data.iloc[0].values], 
                                         columns=gene_data.columns)
        print(f"Restructured clinical features with shape: {clinical_features.shape}")
    else:
        # If the column exists, transpose to have traits as rows
        clinical_features = clinical_data.set_index(trait).T
        clinical_features = pd.DataFrame(index=[trait], data=[clinical_features.iloc[0].values], 
                                         columns=gene_data.columns)
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # Create a DataFrame with all samples classified as cancer (trait = 1)
    # Since the sample characteristics indicated all are gastric cancer
    clinical_features = pd.DataFrame(index=[trait], data=[[1.0] * len(gene_data.columns)], 
                                    columns=gene_data.columns)
    print(f"Created default clinical features with shape: {clinical_features.shape}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values systematically
# Since we know trait values are all 1, create a direct column
linked_data[trait] = 1.0  # Explicitly add the trait column
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Evaluate bias in trait and demographic features
is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation and save metadata
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,
    is_trait_available=True,  # We determined earlier that trait data is available (all samples are cancer)
    is_biased=is_trait_biased, 
    df=linked_data,
    note="Dataset contains gene expression data from gastric cancer samples. All samples are cancer (trait=1)."
)

# 6. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Normalized gene data shape: (20259, 102)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']
Loaded clinical data with shape: (1, 1)
Error in processing clinical data: 102 columns passed, passed data had 1 columns
Created default clinical features with shape: (1, 102)
Linked data shape: (102, 20260)


Data shape after handling missing values: (102, 20260)
Quartiles for 'Stomach_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Stomach_Cancer' in this dataset is severely biased.

Data quality check result: Not usable
Data quality check failed. The dataset is not suitable for association studies.
